### Importación de datos



In [2]:
import pandas as pd

url1 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

tienda1 = pd.read_csv(url1)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

tienda1.head()

,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon
0,Asistente virtual,Electrónicos,164300.0,6900.0,16/01/2021,Pedro Gomez,Bogotá,4,Tarjeta de crédito,8,4.60971,-74.08175
1,Mesa de comedor,Muebles,192300.0,8400.0,18/05/2022,Beatriz Morales,Medellín,1,Tarjeta de crédito,4,6.25184,-75.56359
2,Juego de mesa,Juguetes,209600.0,15900.0,15/03/2021,Juan Fernandez,Cartagena,1,Tarjeta de crédito,1,10.39972,-75.51444
3,Microondas,Electrodomésticos,757500.0,41000.0,03/05/2022,Juan Fernandez,Cali,4,Nequi,1,3.43722,-76.52250
4,Silla de oficina,Muebles,335200.0,20200.0,07/11/2020,Maria Alfonso,Medellín,5,Nequi,1,6.25184,-75.56359


#1. Análisis de facturación



In [3]:
tienda1.shape

(2359, 12)

In [4]:
#Estimando ingresos de cada tienda
ingresos_tienda1 = tienda1['Precio'].sum()
ingresos_tienda2 = tienda2['Precio'].sum()
ingresos_tienda3 = tienda3['Precio'].sum()
ingresos_tienda4 = tienda4['Precio'].sum()

In [5]:
#Dataframe ingreso total en tiendas

df_ingresosTotales = pd.DataFrame({'Ingresos Totales':[ingresos_tienda1,ingresos_tienda2,ingresos_tienda3,ingresos_tienda4]},index=['Tienda 1','Tienda 2','Tienda 3','Tienda 4'])
df_ingresosTotales["Ingresos Totales"] = df_ingresosTotales["Ingresos Totales"].astype(int)
print(df_ingresosTotales)

          Ingresos Totales
Tienda 1        1150880400
Tienda 2        1116343500
Tienda 3        1098019600
Tienda 4        1038375700


# 2. Ventas por categoría

In [6]:
#obteniendo las diferentes categorias de articulos vendidos
categorias_tienda1 = set(tienda1["Categoría del Producto"].unique())
categorias_tienda2 = set(tienda2["Categoría del Producto"].unique())
categorias_tienda3 = set(tienda3["Categoría del Producto"].unique())
categorias_tienda4 = set(tienda4["Categoría del Producto"].unique())

#Uniendo los sets en uno solo
categorias_tiendas = categorias_tienda1|categorias_tienda2|categorias_tienda3|categorias_tienda4

#Creando el DataFrame base
df_tiendas = pd.DataFrame({'Categoría del Producto':list(categorias_tiendas)})


In [7]:
#Funcion pequeña para obtener el conteo de una tienda
def obtener_conteo(df_tienda,nombre_tienda):
  return df_tienda["Categoría del Producto"].value_counts().reset_index().rename(columns = {'count':nombre_tienda,})

#Se une cada conteo al dataframe final
df_tiendas = pd.merge(df_tiendas,obtener_conteo(tienda1,"Tienda 1"),on="Categoría del Producto",how="left")
df_tiendas = pd.merge(df_tiendas,obtener_conteo(tienda2,"Tienda 2"),on="Categoría del Producto",how="left")
df_tiendas = pd.merge(df_tiendas,obtener_conteo(tienda3,"Tienda 3"),on="Categoría del Producto",how="left")
df_tiendas = pd.merge(df_tiendas,obtener_conteo(tienda4,"Tienda 4"),on="Categoría del Producto",how="left")

#Llenamos los huecos donde la tienda no tiene esa categoria con cero
df_tiendas = df_tiendas.fillna(0)
df_tiendas


,Categoría del Producto,Tienda 1,Tienda 2,Tienda 3,Tienda 4
0,Deportes y diversión,284,275,277,277
1,Juguetes,324,313,315,338
2,Instrumentos musicales,182,224,177,170
3,Electrónicos,448,422,451,451
4,Electrodomésticos,312,305,278,254
5,Libros,173,197,185,187
6,Muebles,465,442,499,480
7,Artículos para el hogar,171,181,177,201


# 3. Calificación promedio de la tienda


In [8]:
tienda1['Tienda'] = 'Tienda 1'
tienda2['Tienda'] = 'Tienda 2'
tienda3['Tienda'] = 'Tienda 3'
tienda4['Tienda'] = 'Tienda 4'

df_total_tiendas = pd.concat([tienda1,tienda2,tienda3,tienda4])

tienda_calificaciones = df_total_tiendas.pivot_table(index='Categoría del Producto',columns='Tienda',values='Calificación',aggfunc='mean')

promedio_general = tienda_calificaciones.mean()


tienda_calificaciones.loc['Promedio General'] = promedio_general

pd.options.display.float_format = '{:.2f}'.format

tienda_calificaciones.style.set_caption("Satisfacción del cliente con los productos vendidos")



Tienda,Tienda 1,Tienda 2,Tienda 3,Tienda 4
Categoría del Producto,,,,
Artículos para el hogar,3.748538,4.071823,4.073446,4.009950
Deportes y diversión,3.897887,4.065455,3.938628,4.054152
Electrodomésticos,4.032051,4.072131,4.010791,3.929134
Electrónicos,4.004464,3.971564,3.986696,4.006652
Instrumentos musicales,4.049451,4.035714,4.005650,3.911765
Juguetes,4.077160,4.025559,4.196825,3.988166
Libros,3.976879,3.949239,4.048649,4.042781
Muebles,3.946237,4.092760,4.098196,3.997917
Promedio General,3.966583,4.035531,4.044860,3.992564


In [9]:
# 1. Estilo resaltando el máximo por columna en verde
# y también resaltamos el mínimo en un color suave para contrastar
reporte_con_estilo = tienda_calificaciones.style.highlight_max(axis=0, color='#90EE90') \
                                                .highlight_min(axis=0, color='#FFCCCB') \
                                                .format("{:.2f}")
# 2. Mostrar el reporte
reporte_con_estilo

Tienda,Tienda 1,Tienda 2,Tienda 3,Tienda 4
Categoría del Producto,,,,
Artículos para el hogar,3.75,4.07,4.07,4.01
Deportes y diversión,3.90,4.07,3.94,4.05
Electrodomésticos,4.03,4.07,4.01,3.93
Electrónicos,4.00,3.97,3.99,4.01
Instrumentos musicales,4.05,4.04,4.01,3.91
Juguetes,4.08,4.03,4.20,3.99
Libros,3.98,3.95,4.05,4.04
Muebles,3.95,4.09,4.10,4.00
Promedio General,3.97,4.04,4.04,3.99


In [10]:
# axis=None busca en toda la matriz el número más grande
tienda_calificaciones.style.highlight_max(axis=None, color='yellow').format("{:.2f}")


Tienda,Tienda 1,Tienda 2,Tienda 3,Tienda 4
Categoría del Producto,,,,
Artículos para el hogar,3.75,4.07,4.07,4.01
Deportes y diversión,3.90,4.07,3.94,4.05
Electrodomésticos,4.03,4.07,4.01,3.93
Electrónicos,4.00,3.97,3.99,4.01
Instrumentos musicales,4.05,4.04,4.01,3.91
Juguetes,4.08,4.03,4.20,3.99
Libros,3.98,3.95,4.05,4.04
Muebles,3.95,4.09,4.10,4.00
Promedio General,3.97,4.04,4.04,3.99


# 4. Productos más y menos vendidos

In [11]:
#Lista de tiendas
tiendas = ['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4']
resultados = []

for tienda in tiendas:
  #Obtener el indice del valor maximo y minimo de cada tienda
  idx_max = df_tiendas[tienda].idxmax()
  idx_min = df_tiendas[tienda].idxmin()
  #Se extrae el nombre de la categoria
  cat_max = df_tiendas.loc[idx_max,'Categoría del Producto']
  val_max = df_tiendas.loc[idx_max,tienda]

  cat_min = df_tiendas.loc[idx_min,'Categoría del Producto']
  val_min = df_tiendas.loc[idx_min,tienda]

  resultados.append({
      'Tienda':tienda,
      'Mas vendido':f"{cat_max} ({val_max})",
      'Menos vendido':f"{cat_min} ({val_min})"
  })

#Crear dataFrame con los resultados
df_resultados = pd.DataFrame(resultados)
df_resultados

,Tienda,Mas vendido,Menos vendido
0,Tienda 1,Muebles (465),Artículos para el hogar (171)
1,Tienda 2,Muebles (442),Artículos para el hogar (181)
2,Tienda 3,Muebles (499),Instrumentos musicales (177)
3,Tienda 4,Muebles (480),Instrumentos musicales (170)


# 5. Envío promedio por tienda

In [12]:
# Se crea la tabla sumando los costos de envío
# Usamos aggfunc='sum' para obtener el total de envío por categoría
tiendas_envios = df_total_tiendas.pivot_table(
    index='Categoría del Producto',
    columns='Tienda',
    values='Costo de envío',
    aggfunc='sum'
)

# Se calcula el promedio por cada tienda
# Esto da el costo de envío promedio considerando todas las categorías
promedio_envios = tiendas_envios.mean()

# Se agrega la fila de "Promedio General" al final
tiendas_envios.loc['Promedio General'] = promedio_envios

# Se visualizan el costo más alto usando un color naranja para los costos
tiendas_envios.style.highlight_max(axis=0, color='#FFE4B5').set_caption("SUMA DE COSTOS DE ENVÍO Y PROMEDIO GENERAL").format('{:.2f}')


Tienda,Tienda 1,Tienda 2,Tienda 3,Tienda 4
Categoría del Producto,,,,
Artículos para el hogar,643800.00,800800.00,812400.00,854100.00
Deportes y diversión,2121100.00,1877300.00,1923100.00,1770100.00
Electrodomésticos,19324000.00,18555600.00,17494200.00,15022500.00
Electrónicos,22963700.00,21814200.00,21855900.00,21828300.00
Instrumentos musicales,4883900.00,5557700.00,4108800.00,3953900.00
Juguetes,979500.00,887600.00,1041700.00,1112500.00
Libros,467000.00,555800.00,496400.00,540700.00
Muebles,9994900.00,9436100.00,10784100.00,10235300.00
Promedio General,7672237.50,7435637.50,7314575.00,6914675.00
